# Buckling Beams Problem: Introduction

**Summary**: we consider a beam that is attached to a base on the bottom, and then prescribe a displacement to the top of the beam. For a small displacement, the beam will compress but not bend. However, when the displacement reaches a critical value, the system becomes unstable and the beam will buckle, which will make it bend to either the left side or right side; either option is equally likely.

**Description of the system**: The beam consists of $n$ compressible beam elements, of which element $i$ has length $L_i$ and is compressible with a stiffness $C_i$. There are torsion springs between the base and the first element and between subsequent elements; these torsion springs have spring constant $K_i$. 

![title](beamdiagram.png)

When the beam deforms, the amount element $i$ is compressed is described by the strain $\epsilon_i$. We use Hencky strains, which are defined as $\epsilon_i = \ln\left(\frac{L_i'}{L}\right)$, where $L_i$ is the reference length of the beam element and $L_i'$ the length of the deformed beam element.
The angle the elements make with the vertical are $q_i$. In the undeformed reference configuration the beam is oriented straight up, and therefore these angles are initially all zero.
The displacement prescribed to the top of the beam in downward direction is $d$. It takes a certain unknown force $P$ to apply this displacement.

**Problem**:
* Input: displacement $d$ (applied in increments)
* Parameters: lengths $L_i$, spring constants $K_i$, stiffnesses $C_i$.
* Output: angles $q_i$, strains $\epsilon_i$, optionally force $P$. This output can also be described by the final positions of nodes location at the top and bottom of the beam as well as the joints between elements.

The strain energy is defined as:
$$U(\mathbf{q}, \epsilon) = \frac{1}{2}\left[K_1q_1^2 + \sum_{i=2}^n K_i(q_i-q_{i-1})^2\right] + \frac{1}{2}\sum_{i=1}^n L_iC_i \epsilon_i^2$$

The applied displacement $d$ to the top of the beam is related to the angles $q_i$ and strains $\epsilon_i$ as follows:
$$d = \sum_{i=1}^n L_i \left[1-\exp(\epsilon_i)\cos q_i\right]$$

In BucklingBeams_createDataset.ipynb, we solve this problem by finding the values of $q_i$ and $\epsilon_i$ that minimize the strain energy, while still respecting the constraint from $d$. We do this using a Lagrange multiplier.
We create a dataset by sampling various values of $n$, $K_i$, $C_i$ and $L_i$.
Because the displacement $d$ is applied in increments, this results in trajectories of the beam deforming, where in the final time step $d = \sum_i L_i$, such that the tip of the beam is bent all the way to the base. We created 999 such trajectories.



In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

# The Dataset
Each beam is represented by a path graph, where nodes are located at the top and bottom of the beam as well as the joints between elements, and beam elements are the edges between these nodes. We are using pytorch geometric Data objects to store these graphs.
Let's take a look at the first five graphs in the training dataset.

In [ ]:
with open('../data/BucklingBeams_data.pkl', 'rb') as f:
    data = pickle.load(f)
for graph in data['data_tr'][:5]:
    print(graph)

Each graph has the following attributes with the following shapes:
* edge_index: indicates which nodes are connected by edges. Shape [2, E]
* edge_attr: the edge features. The first feature is the length of the beam element, the second its stiffness under compression C. Shape [E, 2]
* pos: the position of each node for each time step of its trajectory. There are two possible solutions. Shape [n+1, T, dim, n_sol] = [n+1, 200, 2, 2]
* node_attr: the node features. The only feature for each node is the spring constant K of the rotational spring. The node at the top has no spring, so this value is zero. Shape [n+1, 1]
* d: the applied displacement at the top. Shape: [1, T, 1]
* N: contains the number of beam elements $n$. Shape: [1, 1]
* lamb: $\lambda$, which is the Lagrange multiplier that gives the force $P$ needed to force the beam down by displacement $d$. Shape [1, T, n_sol, 1] = [1, 200, 2, 1]

Here $n$ is the number of beam elements, E is the number of edges, which is $2n$, T is the number of time steps (always 200), n_sol is the number of solutions (always 2, although before buckling they will be the same), dim is the number of dimensions (always 2 because we work in 2D).

BucklingBeams_data_processed.pkl: each graph has the following attributes with the following shapes:
* edge_index: indicates which nodes are connected by edges. Shape [2, E]
* edge_attr: the edge features. The first feature is the length of the beam element, the second its stiffness under compression C. Shape [E, 2]
* pos: the position of each node for each time step of its trajectory. There are two possible solutions. Shape [n+1, dim, T, n_sols] = [n+1, 200, 2, 2]
* node_attr: the node features. The only feature for each node is the spring constant K of the rotational spring. The node at the top has no spring, so this value is zero. Shape [n+1, 1]
* d: the applied displacement at the top. Shape: [1, T]
* N: contains the number of beam elements $n$. Shape: [1,]
* lamb: $\lambda$, which is the Lagrange multiplier that gives the force $P$ needed to force the beam down by displacement $d$. Shape [1, T] = [1, 200]

Here $n$ is the number of beam elements, E is the number of edges, which is $2n$, T is the number of time steps (always 200), n_sol is the number of solutions (always 2, although before buckling they will be the same), dim is the number of dimensions (always 2 because we work in 2D).

BucklingBeams_data_fullyConnected.pkl: similar to BucklingBeams_processed.pkl, but with fully connected graphs.

# A plot that shows the beam deforming
The following plot shows show the beam deforms over the trajectory, for both solutions. The gray beam in the middle shows the initial position of the beam, the red dots show where the beam elements are joined with a torsion spring. The size of the red dot indicates the spring constant of the spring, such that a bigger dot means the beam is less likely to bend there.

In [ ]:
# take the first graph from the training data
graph = data['data_tr'][0]

def arr1D_to_str(arr): # for pretty printing
    return '[' + ', '.join([f'{elem:.3f}' for elem in arr]) + ']'

# Extract necessary data from the graph
pos = graph.pos
N = graph.N[0, 0]
d = graph.d[0, :, 0]
K = graph.node_attr[:, 0]
C = graph.edge_attr[:N, 1]
L = graph.edge_attr[:N, 0]
L_tot = pos[-1, 0, 1, 0]  # y-coordinate of last node in first time step

# Set margin
margin = 0.2
factor = margin + 1.0

# Define number of plots and create figure
N_pl = 9  # number of plots
Nsqrt = np.ceil(np.sqrt(N_pl)).astype(int)
fig, axes = plt.subplots(Nsqrt, Nsqrt, figsize=(10, 10))

# Choose indices to plot
inds = np.linspace(0, len(d)-1, N_pl).astype(int)
axes = axes.flatten()

# Iterate over time steps to plot
for ind, ax in zip(inds, axes):
    # Plot initial position
    x, y = pos[:, 0, 0, 0], pos[:, 0, 1, 0]  # select: all nodes, first time step, x and y coordinates, first solution
    ax.plot(x, y, color='lightgray')
    ax.scatter(x, y, s=K*20, color='lightgray')

    # Create rectangle patch for the base
    patch = plt.Rectangle((-L_tot*factor, -L_tot*factor),
                      2*L_tot*factor, L_tot*factor,
                      facecolor='gray', linewidth=0)
    ax.add_patch(patch)

    # Set axes limits and aspect ratio
    ax.set_xlim(-L_tot*factor, L_tot*factor)
    ax.set_ylim(-L_tot*factor*0.1, L_tot*factor)
    ax.set_aspect('equal')

    # Plot all solutions for this time step
    for i in range(pos.shape[2]):
        x, y = pos[:, ind, 0, i], pos[:, ind, 1, i]  # select: all nodes, time step ind, x and y coordinates, solution i
        ax.plot(x, y, color='black')
        ax.scatter(x, y, s=K*20, color='tab:red', zorder=2)

    ax.set_title(f'd={d[ind]:.2f}')

# Widen margins between plots
fig.subplots_adjust(top=0.85, wspace=0.2, hspace=0.35)
fig.suptitle(f'$N={len(C)}$,\n$K={arr1D_to_str(K)}$,\n$C={arr1D_to_str(C)}$,\n$L={arr1D_to_str(L)}$')
plt.show()